<a href="https://colab.research.google.com/github/DarkEol/AutoML-Ontology/blob/main/AutoSki_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#script for extraction of meta-features from datasets along with information about selected algorithm

In [ ]:
!pip install auto-sklearn

In [ ]:
#script for extraction of meta-features 
import autosklearn.classification
import sklearn.model_selection
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

empty_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
if __name__ == '__main__':
  mfeatures = pd.DataFrame(columns=["Name", "NumberOfInstances", "NumberOfFeatures", "ManyInstances", "FewInstances", "ManyFeatures", "FewFeatures", "BinaryClass",	"DateClass", "MissingClassValues", "NominalClass", "NumericClass", "NoClass",	"UnaryClass", "EmptyNominalClass", "StringClass", "RelationalClass", "OnlyMulti-InstanceData", "BinaryAttributes", "DateAttributes", "EmptyNominalAttributes", "MissingValues", "NominalAttributes", "NumericAttributes", "UnaryAttibutes",	"RelationalAttributes",	"StringAttributes", "Algorithm", "Accuracy"])
 
  datasets_nums = [1480, 1485, 1487, 1489, 1491, 1494, 1497]
  i=0

  for dset_num in datasets_nums:
    opml = fetch_openml(data_id=dset_num)
    #data = opml.data
    X = opml.data
    y = opml.target
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=1)

    name = opml.details['name']
    print("name:", name)
    print("X size: ", len(X))
    print("X train size", len(X_train))
    print("X test size: ", len(X_test))

    #target = opml.target
    shape = X.shape
    n_instances = shape[0]
    n_features = shape[1]
    n_target_values = len(y.unique())

    numeric_data = X.select_dtypes(include=[np.number])
    #print("num_data:",numeric_data.shape[1])
    not_numeric_data = X.select_dtypes(exclude=[np.number])
    #print("not_num_data:",not_numeric_data.shape[1])
    category_data = X.select_dtypes(include=['category'])
    #print("category_data:",category_data.shape[1])
    
    mfeatures.loc[len(mfeatures)] = empty_list #adding row 

    mfeatures.at[i,'Name'] = name
    mfeatures.at[i,'NumberOfInstances'] = n_instances
    mfeatures.at[i,'NumberOfFeatures'] = n_features
  
    if n_instances > 3200:
      mfeatures.at[i,'ManyInstances'] = 1
    else:
      mfeatures.at[i,'FewInstances'] = 1
  
    if n_features > 100:
      mfeatures.at[i,'ManyFeatures'] = 1
    else:
      mfeatures.at[i, 'FewFeatures'] = 1

    if n_target_values == 2:
      mfeatures.at[i, 'BinaryClass'] = 1
      print("Binary")
    else:
      print("MultiClass")

    if y.str.isnumeric().any():
      mfeatures.at[i, 'NumericClass'] = 1
    else:
      mfeatures.at[i, 'StringClass'] = 1

    if y.dtype=='category':
      mfeatures.at[i, 'NominalClass'] = 1

    if numeric_data.shape[1] > 0:
      mfeatures.at[i, 'NumericAttributes'] = 1
  
    if category_data.shape[1] > 0:
      mfeatures.at[i, 'NominalAttributes'] = 1

    for fname in opml.feature_names:
      if len(X[fname].unique())==2: # and not binary_attributes:
        mfeatures.at[i, 'BinaryAttributes'] = 1

      if len(X[fname].unique())==1:
        mfeatures.at[i, 'UnaryAttibutes'] = 1

      if X[fname].isna().sum() > 0:
        mfeatures.at[i, 'MissingValues'] = 1

      if not pd.api.types.is_numeric_dtype(X[fname]):
        mfeatures.at[i, 'StringAttributes'] = 1

    #AutoML settings
    cls = autosklearn.classification.AutoSklearnClassifier(ensemble_class=None, time_left_for_this_task=600)

    #running search for best model
    print("searching for model")
    cls.fit(X_train, y_train)
    print("model found")

    #info about found models
    #print("leaderboard")
    #print(cls.leaderboard())
    algo = cls.leaderboard().iat[0,2]
    score = cls.score(X_test, y_test)

    print("algo: ",algo)
    print("score: ",score)
    mfeatures.at[i, 'Algorithm'] = algo
    mfeatures.at[i, 'Accuracy'] = score
    i+=1

In [ ]:
print(type(cls.leaderboard()))
print(cls.leaderboard())
print(cls.leaderboard().columns)

algo = cls.leaderboard().iat[0,2]
score = cls.score(X_test, y_test)

print("algo: ",algo)
print("score: ",score)
print(mfeatures)

In [ ]:
print(mfeatures[['Name','ManyInstances', 'FewInstances', 'ManyFeatures', 'FewFeatures', 'BinaryClass','NumericClass','StringClass','NominalClass','BinaryAttributes','MissingValues','NominalAttributes','NumericAttributes','StringAttributes','UnaryAttibutes','Algorithm','Accuracy']])
print(mfeatures.shape)
mfeatures.to_csv('features.csv')
mfeatures.to_excel('features.xlsx')